In [1]:
import sqlparse

from cast.models.pages import PostData
from cast.feeds import LatestEntriesFeed

from django.test.client import RequestFactory
from django.db import connection, reset_queries

In [2]:
def count_queries(func):
    reset_queries()
    response = func()
    print(len(response.content))
    return len(connection.queries)


def show_queries(queries):
    for query in queries:
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        print(formatted_sql)    

In [3]:
def get_all_data_for_posts(post_queryset):
        queryset = post_queryset
        queryset = queryset.select_related("owner")
        queryset = queryset.prefetch_related(
            "audios",
            "images",
            "videos",
            "galleries",
            "galleries__images",
            "images__renditions",
            "galleries__images__renditions",
        )
        return queryset 

In [4]:
def count_latest_entries_feed_queries(blog_slug):
    reset_queries()
    feed_view = LatestEntriesFeed()
    request_factory = RequestFactory()
    request = request_factory.get(f"/blogs/{blog_slug}/feed/rss.xml")
    response = feed_view(request, slug=blog_slug)
    print(len(connection.queries))    

In [5]:
%%time
# count_latest_entries_feed_queries("ephes_blog")

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11 µs


In [6]:
blog_slug = "ephes_blog"
blog = Blog.objects.get(slug=blog_slug)
post_queryset = Post.objects.live().descendant_of(blog).order_by("-visible_date")

### Just all Posts

In [7]:
%%time
reset_queries()
post_list = list(get_all_data_for_posts(post_queryset))
#renditions = list(Post.get_all_renditions_from_queryset(post_list))
print(len(connection.queries))

8
CPU times: user 122 ms, sys: 3.98 ms, total: 126 ms
Wall time: 144 ms


### All Posts rendered description

In [8]:
def blocker(*args):
    raise Exception("No database access allowed here.")

In [9]:
import wagtail

from unittest import mock

from wagtail.rich_text.pages import PageLinkHandler

In [14]:
def build_cached_get_instance(page_cache):
    @classmethod
    def cached_get_instance(cls, attrs):
        return page_cache[attrs["id"]]
    
    return cached_get_instance

linked_page = Page.objects.get(pk=444).specific
print(linked_page.url)
new_get_instance = build_cached_get_instance({str(linked_page.pk): linked_page})

with mock.patch.object(wagtail.rich_text.pages.PageLinkHandler, 'get_instance', new_get_instance):
    foo = PageLinkHandler.get_instance({"id": "444"})
print(foo)

/blogs/ephes_blog/upgrading-postgres/
Upgrading Postgres


In [16]:
foo

<Post: Upgrading Postgres>

In [17]:
linked_page = Page.objects.get(pk=444).specific

In [18]:
linked_page.url

'/blogs/ephes_blog/upgrading-postgres/'

In [19]:
%%time
blog_slug = "das_claas_und_nora_blog"
blog_slug = "ephes_blog"
blog = Blog.objects.get(slug=blog_slug)
post_queryset = Post.objects.live().descendant_of(blog).filter(pk__in=[438]).order_by("-visible_date")
post_list = get_all_data_for_posts(post_queryset)
renditions_for_posts = Post.get_all_renditions_from_queryset(post_list)
images = {}
for post in post_list:
    for image_type, image in post.get_all_images():
        images[image.pk] = image
post_data = PostData(
    blog=blog, renditions_for_posts=renditions_for_posts, template_base_dir="bootstrap5",
    images=images
)
request_factory = RequestFactory()
request = request_factory.get(f"/blogs/{blog_slug}/feed/rss.xml")
reset_queries()
with mock.patch.object(wagtail.rich_text.pages.PageLinkHandler, 'get_instance', new_get_instance):
    for post in post_list:
        with connection.execute_wrapper(blocker):
            print("post: ", post.pk)
            post.description = post.get_description(
                request=request, render_detail=True, escape_html=False, post_data=post_data
            )
print(len(connection.queries))

TypeError: PostData.__init__() missing 10 required keyword-only arguments: 'site', 'blog_url', 'post_by_id', 'root_nav_links', 'has_audio_by_id', 'page_url_by_id', 'owner_username_by_id', 'videos', 'audios', and 'audios_by_post_id'

In [15]:
print(len(connection.queries))

20


In [16]:
len(connection.queries)

20

In [17]:
show_queries(connection.queries)

SELECT "wagtailcore_page"."id",
       "wagtailcore_page"."path",
       "wagtailcore_page"."depth",
       "wagtailcore_page"."numchild",
       "wagtailcore_page"."translation_key",
       "wagtailcore_page"."locale_id",
       "wagtailcore_page"."latest_revision_id",
       "wagtailcore_page"."live",
       "wagtailcore_page"."has_unpublished_changes",
       "wagtailcore_page"."first_published_at",
       "wagtailcore_page"."last_published_at",
       "wagtailcore_page"."live_revision_id",
       "wagtailcore_page"."go_live_at",
       "wagtailcore_page"."expire_at",
       "wagtailcore_page"."expired",
       "wagtailcore_page"."locked",
       "wagtailcore_page"."locked_at",
       "wagtailcore_page"."locked_by_id",
       "wagtailcore_page"."title",
       "wagtailcore_page"."draft_title",
       "wagtailcore_page"."slug",
       "wagtailcore_page"."content_type_id",
       "wagtailcore_page"."url_path",
       "wagtailcore_page"."owner_id",
       "wagtailcore_page"."seo_title"

In [18]:
#dir(post.body)

In [19]:
foo = None
for block in post.get_all_blocks(post.body):
    print("block: ", block.block_type)
    if block.block_type == "image":
        foo = block

AttributeError: 'Post' object has no attribute 'get_all_blocks'

In [20]:
block = foo

In [21]:
dir(block)

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [22]:
dir(block.block)

AttributeError: 'NoneType' object has no attribute 'block'

In [23]:
block = block.block

AttributeError: 'NoneType' object has no attribute 'block'

In [24]:
block.value_from_datadict()

AttributeError: 'NoneType' object has no attribute 'value_from_datadict'

In [25]:
block.extract_references?

Object `block.extract_references` not found.


In [26]:
post = Post.objects.get(pk=438)

In [27]:
post.body.raw_data

[{'id': '591b2ead-c1bb-49a9-8556-e744fde7a53d', 'type': 'overview', 'value': [{'id': '0c9b1e29-8f60-42cc-97f3-9d13f89673dd', 'type': 'paragraph', 'value': '<p data-block-key="9o8kt">At work I used <a href="https://htmx.org/">htmx</a> and <a href="https://alpinejs.dev/">Alpine.js</a> again and I think this is a really powerful combination. I hope the <a href="https://developer.chrome.com/docs/web-platform/view-transitions/">view transition api</a>, which is currently only implemented in Chrome, will be available in other browsers soon. I also did some thinking about how to display PDFs on web pages using <a href="https://mozilla.github.io/pdf.js/">PDF.js</a>.</p><p data-block-key="8g74q"></p><p data-block-key="6umfe">For my personal site, I had to fix the image rendition deletion incident from last week, and I thought a bit about <a href="https://github.com/ephes/wagtail_srcset">wagtail_srcset</a> and new image formats like <a href="https://en.wikipedia.org/wiki/AVIF">AVIF</a>. Maybe I 

In [28]:
post.get_site().root_page.get_children().live()

<PageQuerySet [<Page: Claas und Nora sagen Hallo>, <Page: Ephes Blog>]>

In [29]:
Post()

<Post: >

In [30]:
post = Post.objects.last()

In [31]:
post.body.raw_data

[{'id': '681a02b8-95b2-48ec-859d-7a765a357335', 'type': 'overview', 'value': [{'id': 'af2399c8-908f-4f33-9a06-b59214f0cf4f', 'type': 'paragraph', 'value': '<p data-block-key="fnhbp">Das ist ein kleiner post</p>'}, {'id': '2cd83eb2-aac0-4dd4-9475-f3f5245f701c', 'type': 'image', 'value': 2255}, {'id': 'b706ccca-e8af-425b-9d03-0cbccb2d779e', 'type': 'gallery', 'value': {'layout': 'default', 'gallery': [{'id': '8937f20e-9ed8-470c-93a4-9a0e38ef9f20', 'type': 'item', 'value': 2253}, {'id': 'c87421ec-c7cb-44d5-90e8-6424bd44d36b', 'type': 'item', 'value': 2256}]}}]}]

In [32]:
post.revisions

<django.contrib.contenttypes.fields.create_generic_related_manager.<locals>.GenericRelatedObjectManager at 0x156ca0d40>

In [33]:
post.has_unpublished_changes

False

In [34]:
revision = post.revisions.last()

In [35]:
#dir(revision)

In [36]:
post.publish(revision)

bulk_to_python for cast image:  [2255]
self model class:  <class 'wagtail.images.models.Image'>
media model:  2255 image <class 'int'>
media model:  Gallery object (349) gallery <class 'cast.models.gallery.Gallery'>


In [37]:
post.body.raw_data

[{'id': '681a02b8-95b2-48ec-859d-7a765a357335', 'type': 'overview', 'value': [{'id': 'af2399c8-908f-4f33-9a06-b59214f0cf4f', 'type': 'paragraph', 'value': '<p data-block-key="fnhbp">Das ist ein kleiner post</p>'}, {'id': '2cd83eb2-aac0-4dd4-9475-f3f5245f701c', 'type': 'image', 'value': 2255}, {'id': 'b706ccca-e8af-425b-9d03-0cbccb2d779e', 'type': 'gallery', 'value': {'layout': 'default', 'gallery': [{'id': '8937f20e-9ed8-470c-93a4-9a0e38ef9f20', 'type': 'item', 'value': 2253}, {'id': 'c87421ec-c7cb-44d5-90e8-6424bd44d36b', 'type': 'item', 'value': 2256}]}}]}]